# Dask DataFrame

Материалы: 
* Макрушин С.В. Лекция 13: Dask DataFrame
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считать данные из файлов в каталоге `accounts`. Содержат ли какие-либо из столбцов пропуски?

2. Подсчитать количество раз, которое то или иное имя встретилось в выборке. Вывести самое часто встречающееся имя.

3. Создать новую колонку, которая является результатом от деления значения `amount` нацело на 100, если `amount` > 100, и нулём в противном случае.

## Лабораторная работа 13

__При решении данных задач не подразумевается использования других коллекций, кроме `dask.DataFrame`, если в задании явно не указано обратное.__

In [2]:
import dask.dataframe as dd
import pandas as pd
import dask.bag as db
import json

1. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты.

In [3]:
recipes = dd.read_csv(
    './recipes_full/recipes_*.csv',
    dtype={'minutes': 'float64', 'n_steps': 'float64'},
    parse_dates=True
)
recipes["submitted"] = dd.to_datetime(recipes['submitted'])
recipes = recipes.set_index('id')

2. Выведите метаинформацию о таблице: `npartitions` и типы столбцов.

In [51]:
dir(recipes)

['__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_ufunc__',
 '__array_wrap__',
 '__await__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__dask_graph__',
 '__dask_keys__',
 '__dask_layers__',
 '__dask_optimize__',
 '__dask_postcompute__',
 '__dask_postpersist__',
 '__dask_scheduler__',
 '__dask_tokenize__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__iter__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__

In [5]:
recipes

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
npartitions=24,,,,,,,
38,object,float64,int64,datetime64[ns],float64,object,int64
190639,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
2120964,...,...,...,...,...,...,...
2231636,...,...,...,...,...,...,...


In [52]:
recipes.describe()

,minutes,contributor_id,n_steps,n_ingredients,month
npartitions=1,,,,,
,float64,float64,float64,float64,float64
,...,...,...,...,...


3. Выведите на экран 5 первых строк таблицы. Выведите на экран 5 последних строк таблицы. В случае сообщения об ошибки объясните причину и исправьте ошибку.

In [6]:
recipes.head()

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
38,low fat berry blue frozen dessert,1485.0,1533,1999-08-09,13.0,"this is yummy and low-fat, it always turns out...",4
39,biryani,265.0,1567,1999-08-29,17.0,"delhi, india",26
40,best lemonade,35.0,1566,1999-09-05,8.0,this is from one of my first good house keepi...,6
41,carina s tofu vegetable kebabs,1460.0,1586,1999-09-03,18.0,this dish is best prepared a day in advance to...,15
43,best blackbottom pie,140.0,34879,1999-08-21,35.0,"sweet, chocolatey, yummy",15


In [7]:
recipes.tail()

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
2231632,zwiebelsuppe soldiers depp somma,13.0,226867,2004-01-08,5.0,this is a very good easy recipe that was so am...,3
2231633,vaisiu kesar pimlico dominick backwoods,37.0,11176,2003-07-27,2.0,posted for zaar world tour 5. from http://www....,4
2231634,chez basico bracha vijs scape uomo marathon pe...,14.0,362919,1997-07-01,5.0,"this is a simple, yet tasty way to make creame...",2
2231635,lumpia tunisia whisper medford monsieur presen...,42.0,60260,2021-03-18,5.0,my family loves fish & chips from the 'chippie...,2
2231636,brodet casava celery kahwe hall bauernsalat al...,16.0,857184,2014-06-23,4.0,"this chicken is filled with mushrooms, onions,...",3


4. Посчитайте, сколько строк содержит каждый из блоков (partitions).

In [8]:
recipes.map_partitions(len).compute()

0     107601
1      46808
2     141880
3       3010
4     208648
5      49676
6       6241
7     156724
8     156804
9      52990
10      5981
11    149991
12    149116
13     67595
14      6179
15    198595
16    158366
17      7303
18     33902
19    222405
20      4366
21    173768
22     13015
23    110673
dtype: int64

5. Найдите максимум в столбце `n_steps`. Визуализируйте граф вычислений для этой задачи. Прокомментируйте логику работы `dask` при решении данной задачи.

In [9]:
maxi = recipes['n_steps'].max(axis=0)

In [10]:
maxi.compute()

145.0

In [13]:
maxi.visualize()  # по идее так но не работает

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

6. Посчитайте количество рецептов с группировкой по месяцам добавления отзыва в базу.

In [14]:
recipes['month'] = recipes['submitted'].map(lambda x: x.month)

In [15]:
count_recipes = recipes.groupby('month').size().compute()
count_recipes

month
1     193363
2     173834
3     192389
4     186049
5     192487
6     184205
7     189337
8     187276
9     181081
10    187018
11    180974
12    183624
dtype: int64

7. Считайте файлы из архива `reviews_full.zip` (__ЛР12__) в виде `dask.bag`. Пользуясь результатом лабораторной работы 12, рассчитайте среднее значение оценок отзывов с группировкой по месяцам. После завершения всех вычислений преобразуйте результат к `pd.Series`.

In [34]:
import re


def add_rating(line):
    res = json.loads(line[0])
    ans = dict()
    ans['rating'] = int(re.findall('reviews_(\d)\.json', line[1])[0])
    ans['month'] = int(res['date'][5:7])
    return ans

In [32]:
reviews = db.read_text(
    './reviews_full/reviews_*.json',
    include_path=True
).map(add_rating)

In [33]:
reviews.take(1)

({'rating': 0, 'month': 5},)

In [40]:
rate_mean = reviews.foldby(
    'month',
    lambda t, x: (t[0] + 1, t[1] + x['rating']),
    (0, 0),
    lambda x, y: (x[0] + y[0], x[1] + y[1])
).compute()

In [50]:
means = [{'month':i[0], 'mean':i[1][1]/i[1][0]} for i in rate_mean]
pd.DataFrame(means)

,month,mean
0,5,4.415401
1,6,4.418419
2,11,4.408829
3,9,4.410661
4,10,4.411452
5,4,4.412259
6,2,4.407135
7,3,4.406911
8,12,4.406908
9,1,4.408553


8. Пользуясь результатами решения задач 6 и 7, создайте `pd.DataFrame`, содержащий два столбца: `mean_rating`, `recipes_count`

In [25]:
res = pd.DataFrame()
res['mean_rating'] = means['mean']
res['recipes_count'] = count_recipes
res.sort_index()

,mean_rating,recipes_count
month,,
1,4.408553,193363
2,4.407135,173834
3,4.406911,192389
4,4.412259,186049
5,4.415401,192487
6,4.418419,184205
7,4.411717,189337
8,4.410822,187276
9,4.410661,181081


In [53]:
d = pd.read_csv('reviews_sample.csv')

In [56]:
d["review"].map(lambda x:)

0         Last week whole sides of frozen salmon fillet ...
1         So simple and so tasty!  I used a yellow capsi...
2         Very nice breakfast HH, easy to make and yummy...
3         These are a favorite for the holidays and so e...
4         Excellent soup!  The tomato flavor is just gre...
                                ...                        
126691    This recipe was great! I made it last night. I...
126692    This recipe is outstanding. I followed the rec...
126693    Well, we were not a crowd but it was a fabulou...
126694    I have been a steak eater and dedicated BBQ gr...
126695    Wonderful and simple to prepare seasoning blen...
Name: review, Length: 126696, dtype: object